# © Artur Czarnecki. All rights reserved.
# Integrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.


In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

### Load files with metadata

In [ ]:
from intergrax.rag.documents_loader import IntergraxDocumentsLoader

import intergrax.logging

doc_loader = IntergraxDocumentsLoader(verbose=True, docx_mode="paragraphs")
docs = doc_loader.load_documents("../documents/mooff-strategy",)
print(f"Loaded docs: {len(docs)}")

### Split documents into chunks

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

from intergrax.rag.documents_splitter import IntergraxDocumentsSplitter
from intergrax.rag.documents_loader import IntergraxDocumentsLoader
from langchain_core.documents import Document
import intergrax.logging

doc_loader = IntergraxDocumentsLoader(verbose=True, docx_mode="paragraphs")
docs = doc_loader.load_documents("../documents/mooff-strategy",)

splitter = IntergraxDocumentsSplitter(verbose=True)
split_docs = splitter.split_documents(documents=docs)

print(f"Loaded docs: {len(docs)} - splitter into {len(split_docs)} chunks")

# Embedding

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

from intergrax.rag.documents_splitter import IntergraxDocumentsSplitter
from intergrax.rag.documents_loader import IntergraxDocumentsLoader
from intergrax.rag.embedding_manager import IntergraxEmbeddingManager
import intergrax.logging

doc_loader = IntergraxDocumentsLoader(verbose=True)
docs = doc_loader.load_documents("../documents/mooff-strategy/doc",)

splitter = IntergraxDocumentsSplitter(verbose=True)
split_docs = splitter.split_documents(documents=docs)

embed_manager = IntergraxEmbeddingManager(
    verbose=True,
    provider="ollama",
    model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16:latest", 
    assume_ollama_dim=1536)

embeddings, documents = embed_manager.embed_documents(docs=docs)
print(f"Embedding length: {len(embeddings)} for documents: {len(docs)}")

# Vectorstore

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

from intergrax.rag.embedding_manager import IntergraxEmbeddingManager
from intergrax.rag.vectorstore_manager import IntergraxVectorstoreManager, VSConfig
from intergrax.rag.documents_splitter import IntergraxDocumentsSplitter
from intergrax.rag.documents_loader import IntergraxDocumentsLoader
from langchain_core.documents import Document
from collections import Counter

TENANT = "intergrax"
CORPUS = "intergrax-strategy"
VERSION = "v1"

# 0) Vectorstore na starcie (bez ładowania/splita/embedu)
cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs",
    chroma_persist_directory="chroma_db/intergrax_docs_v1",
)
store = IntergraxVectorstoreManager(config=cfg, verbose=True)

# 1) Tylko 'probe' do sprawdzenia obecności korpusu (lekko, bez embedowania chunków)
embed_manager = IntergraxEmbeddingManager(
    verbose=False,
    provider="ollama",
    model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16:latest",
    assume_ollama_dim=1536
)

def corpus_present(store: IntergraxVectorstoreManager, embed_mgr: IntergraxEmbeddingManager) -> bool:
    if store.count() == 0:
        return False
    qvec = embed_mgr.embed_one("probe")
    
    where = {
        "$and": [
            {"tenant": {"$eq": TENANT}},
            {"corpus": {"$eq": CORPUS}},
            {"version": {"$eq": VERSION}},
        ]
    }

    res = store.query(query_embeddings=qvec, top_k=1, where=where)
    return bool(res["ids"] and res["ids"][0])

NEED_INGEST = not corpus_present(store, embed_manager)
print(f"Need ingest: {NEED_INGEST}")

if not NEED_INGEST:
    # Nic nie ładujemy: unikamy zbędnego splitu i embedów
    print("[INGEST] Skipping — corpus already present.")
else:
    # 2) Load
    doc_loader = IntergraxDocumentsLoader(verbose=False)
    docs = doc_loader.load_documents("../documents/mooff-strategy/doc")

    # 3) Split + metadata
    def add_meta(chunk_doc: Document, idx: int, total: int):
        return {"tenant": TENANT, "corpus": CORPUS, "version": VERSION}

    splitter = IntergraxDocumentsSplitter(verbose=False)
    split_docs = splitter.split_documents(documents=docs, call_custom_metadata=add_meta)

    # 4) Embed tylko chunków
    embeddings, documents = embed_manager.embed_documents(docs=split_docs)

    # 5) Stabilne ID — bierz prosto z metadanych splittera (chunk_id)
    ids = []
    for d in documents:
        cid = d.metadata.get("chunk_id")
        if not cid:
            # awaryjnie: parent + index (hash już nadaje splitter; tu fallback)
            parent = d.metadata.get("parent_id") or d.metadata.get("source_path") or d.metadata.get("source_name", "doc")
            idx = int(d.metadata.get("chunk_index", 0))
            cid = f"{parent}#ch{idx:04d}"
        ids.append(cid)

    # 6) Deduplikacja ID w paczce (Chroma wymaga unikalnych ID per upsert)
    def dedup_batch(ids, docs, embs):
        seen = set()
        new_ids, new_docs, new_embs = [], [], []
        for i, _id in enumerate(ids):
            if _id in seen:
                continue
            seen.add(_id)
            new_ids.append(_id)
            new_docs.append(docs[i])
            new_embs.append(embs[i])
        return new_ids, new_docs, new_embs

    ids, documents, embeddings = dedup_batch(ids, documents, embeddings)

    # (opcjonalnie) ostrzeż, jeśli w batchu były duplikaty
    c = Counter(ids)
    dups = [k for k, v in c.items() if v > 1]
    if dups:
        print(f"[WARN] Duplicate IDs after dedup? {len(dups)}")

    # 7) Ingest
    base_metadata = {"tenant": TENANT, "corpus": CORPUS, "version": VERSION}
    store.add_documents(
        documents=documents,
        embeddings=embeddings,
        ids=ids,
        batch_size=128,
        base_metadata=base_metadata
    )
    print("[INGEST] Vectorstore updated. Count:", store.count())

### RAG Retriever Test

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

from intergrax.rag.embedding_manager import IntergraxEmbeddingManager
from intergrax.rag.rag_retriever import IntergraxRagRetriever
from intergrax.rag.vectorstore_manager import IntergraxVectorstoreManager, VSConfig

cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs",
    chroma_persist_directory="chroma_db/intergrax_docs_v1",
)
store = IntergraxVectorstoreManager(config=cfg, verbose=False)


embed_manager = IntergraxEmbeddingManager(
    verbose=False,
    provider="ollama",
    model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16:latest",
    assume_ollama_dim=1536
)


retriever = IntergraxRagRetriever(store, embed_manager, verbose=True)

question = "Czym są wirtualne targi mooff ?"

# MMR
hits = retriever.retrieve(
    question=question,
    top_k=8,
    score_threshold=0.15,
    where={"tenant": TENANT, "corpus": CORPUS, "version": VERSION},
    max_per_parent=2,
    use_mmr=True,
    include_embeddings=True,
    prefetch_factor=5
)
for h in hits:
    print(h["rank"], f"{h['similarity_score']:.3f}", h["metadata"])
    print(h['content'])
    print()

# ReRanker Test

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

from intergrax.rag.embedding_manager import IntergraxEmbeddingManager
from intergrax.rag.rag_retriever import IntergraxRagRetriever
from intergrax.rag.vectorstore_manager import IntergraxVectorstoreManager, VSConfig
from intergrax.rag.re_ranker import IntergraxReRanker, ReRankerConfig

cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs",
    chroma_persist_directory="chroma_db/intergrax_docs_v1",
)
store = IntergraxVectorstoreManager(config=cfg, verbose=False)


embed_manager = IntergraxEmbeddingManager(
    verbose=False,
    provider="ollama",
    model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16:latest",
    assume_ollama_dim=1536
)

reranker = IntergraxReRanker(
    embedding_manager=embed_manager,
    config=ReRankerConfig(
        use_score_fusion=True,
        fusion_alpha=0.4,
        normalize="minmax",
        doc_batch_size=256
    ),
    verbose=True
)

retriever = IntergraxRagRetriever(store, embed_manager, verbose=True)

question = "Czym są wirtualne targi mooff ?"

hits = retriever.retrieve(
    question=question,
    top_k=8,
    score_threshold=0.15,
    where={"tenant": TENANT, "corpus": CORPUS, "version": VERSION},
    max_per_parent=2,
    reranker=reranker
)
for h in hits:
    print(h["rank"], f"{h['similarity_score']:.3f}", h["metadata"])
    print(h['content'])
    print()


# RAG with LLM Test

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

from intergrax.rag.embedding_manager import IntergraxEmbeddingManager
from intergrax.rag.rag_retriever import IntergraxRagRetriever
from intergrax.rag.vectorstore_manager import IntergraxVectorstoreManager, VSConfig
from intergrax.rag.re_ranker import IntergraxReRanker, ReRankerConfig
from intergrax.rag.rag_answerer import IntergraxAnswererConfig, IntergraxRagAnswerer
from intergrax.llm_adapters import LLMAdapterRegistry
from langchain_ollama import ChatOllama
import intergrax.system_prompts as prompts

cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs",
    chroma_persist_directory="chroma_db/intergrax_docs_v1",
)
store = IntergraxVectorstoreManager(config=cfg, verbose=False)


embed_manager = IntergraxEmbeddingManager(
    verbose=False,
    provider="ollama",
    model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16:latest",
    assume_ollama_dim=1536
)

reranker = IntergraxReRanker(
    embedding_manager=embed_manager,
    config=ReRankerConfig(
        use_score_fusion=True,
        fusion_alpha=0.4,
        normalize="minmax",
        doc_batch_size=256
    ),
    verbose=True
)

retriever = IntergraxRagRetriever(store, embed_manager, verbose=False)


# create answerer configuration
cfg = IntergraxAnswererConfig(
    top_k=10,
    min_score=0.15,
    re_rank_k=5,
    max_context_chars=12000,
)
cfg.system_instructions = prompts.default_rag_system_instruction()
cfg.system_context_template = "Użyj następującego kontekstu aby odpowiedzieć na pytanie użytkownika: {context}"


llm = LLMAdapterRegistry.create(name="ollama", chat= ChatOllama(model="llama3.1:latest"))


# create rag answerer (retriever + LLM)
answerer = IntergraxRagAnswerer(
    retriever=retriever,
    llm=llm,
    reranker=reranker,  
    config=cfg,
    verbose=False,
)

questions = [
    "Jakie unikalne przewagi na tle konkurencji posiada Mooff? Odpowiedź bardzo dokładnie używając nie mniej niż 1000 słów.",
    "Czym są wirtualne targi na platformie mooff ?",    
]


for question in questions:
    # create answer
    print("QUESTION: ", question)
    res = answerer.run(
        question=question,
        stream=False,
        summarize=True,
    )

    print("ANSWER: ", res['answer'])
    print()

# min wiring : Dual-Index (TOC + Chunks) + map-reduce + citates

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

from intergrax.rag.embedding_manager import IntergraxEmbeddingManager
from intergrax.rag.vectorstore_manager import IntergraxVectorstoreManager, VSConfig
from intergrax.rag.re_ranker import IntergraxReRanker, ReRankerConfig
from intergrax.rag.rag_answerer import IntergraxAnswererConfig, IntergraxfRagAnswerer
from intergrax.rag.rag_answerer import IntergraxAnswererConfig, IntergraxRagAnswerer
from intergrax.llm.conversational_memory import IntergraxConversationalMemory
from intergrax.rag.dual_index_builder import build_dual_index
from intergrax.rag.dual_retriever import IntergraxDualRetriever
from intergrax.rag.windowed_answerer import IntergraxWindowedAnswerer

from intergrax.rag.documents_loader import IntergraxDocumentsLoader
from intergrax.rag.documents_splitter import IntergraxDocumentsSplitter

from langchain_ollama import ChatOllama
import intergrax.system_prompts as prompts

# -----------------------------------------------------------
# 1) Vectorstore: CHUNKS + TOC (ta sama ścieżka persist)
# -----------------------------------------------------------
vs_chunks = IntergraxVectorstoreManager(VSConfig(
    provider="chroma",
    collection_name="intergrax_chunks",
    chroma_persist_directory="chroma_db/intergrax_docs_v1",
))
vs_toc = IntergraxVectorstoreManager(VSConfig(
    provider="chroma",
    collection_name="intergrax_toc",
    chroma_persist_directory="chroma_db/intergrax_docs_v1",
))

# -----------------------------------------------------------
# 2) Embedding manager
# -----------------------------------------------------------
embed_manager = IntergraxEmbeddingManager(
    verbose=False,
    provider="ollama",
    model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16:latest",
    assume_ollama_dim=1536
)

# -----------------------------------------------------------
# 3) Ingest dokumentów i budowa dual indexu
#    Automatycznie uruchamiany tylko, gdy kolekcje są puste
# -----------------------------------------------------------
def safe_count(vs_manager):
    try:
        return int(vs_manager.count() or 0)
    except Exception:
        return 0  # jeśli nie da się policzyć → traktuj jako 0

chunks_count = safe_count(vs_chunks)
toc_count    = safe_count(vs_toc)

if chunks_count == 0:
    print(f"[INFO] Vectorstore CHUNKS empty (TOC={toc_count}) — performing initial ingest...")

    loader = IntergraxDocumentsLoader(
        verbose=True,
        docx_mode="paragraphs",
        pdf_enable_ocr=True,
        image_enable_ocr=True
    )
    raw_docs = loader.load_documents("../documents/mooff-strategy/doc")

    splitter = IntergraxDocumentsSplitter(verbose=True)
    docs = splitter.split_documents(raw_docs)
    print(f"Loaded {len(docs)} documents after splitting")

    build_dual_index(
        docs=docs,
        embed_manager=embed_manager,
        vs_chunks=vs_chunks,
        vs_toc=vs_toc,
        batch_size=512,
        verbose=True,
    )
elif toc_count == 0:
    print("[WARN] CHUNKS exist but TOC is empty — skipping full ingest to avoid duplicates.")
    print("       (Jeśli to niezamierzone, dobuduj TOC osobno lub upewnij się, że loader/splitter oznaczają nagłówki: doc_type='docx', is_heading=True).")
else:
    print(f"[INFO] Vectorstore already populated — CHUNKS={chunks_count}, TOC={toc_count}. Skipping ingest.")


# -----------------------------------------------------------
# 4) Dual retriever + Reranker
# -----------------------------------------------------------
retriever = IntergraxDualRetriever(
    vs_chunks=vs_chunks,
    vs_toc=vs_toc,
    embed_manager=embed_manager,
    k_chunks=40,
    k_toc=10,
    verbose=True,
)

reranker = IntergraxReRanker(
    embedding_manager=embed_manager,
    config=ReRankerConfig(
        use_score_fusion=True,
        fusion_alpha=0.4,
        normalize="minmax",
        doc_batch_size=256
    ),
    verbose=True
)

# -----------------------------------------------------------
# 5) LLM + Answerer (z pamięcią)
# -----------------------------------------------------------
cfg = IntergraxAnswererConfig(
    top_k=20,
    min_score=None,
    re_rank_k=8,
    max_context_chars=12000,
    temperature=0.0,
)
cfg.system_instructions = prompts.default_rag_system_instruction()
cfg.system_context_template = "Użyj następującego kontekstu aby odpowiedzieć na pytanie użytkownika: {context}"

llm = LLMAdapterRegistry.create(name="ollama", chat=ChatOllama(model="llama3.1:latest"))

memory = IntergraxConversationalMemory()

answerer = IntergraxRagAnswerer(
    retriever=retriever,
    llm=llm,
    reranker=reranker,
    config=cfg,
    memory=memory,
    verbose=False,
)

# -----------------------------------------------------------
# 6) Warstwa okienkowa (map→reduce) dla długich odpowiedzi
# -----------------------------------------------------------
windowed = IntergraxWindowedAnswerer(answerer=answerer, retriever=retriever, verbose=True)

# -----------------------------------------------------------
# 7) Zapytania
#    - długie pytanie → tryb windowed (lepsza spójność przy dużym kontekście)
#    - krótsze pytanie → standardowy answerer.ask
# -----------------------------------------------------------
questions = [
    ("standard", "Czym są wirtualne targi na platformie Mooff?"),
    ("windowed", "Jakie unikalne przewagi na tle konkurencji posiada Mooff? Odpowiedź bardzo dokładnie używając nie mniej niż 1000 słów."),    
]

for mode, question in questions:
    print("\n==========================")
    print("QUESTION (", mode, "): ", question)
    if mode == "windowed":
        res = windowed.ask_windowed(
            question=question,
            top_k_total=60,
            window_size=12,
            summarize_each=True,
        )
    else:
        res = answerer.run(
            question=question,
            stream=False,
            summarize=True,
        )

    print("\n=== ANSWER ===\n", res["answer"])
    if res.get("summary"):
        print("\n=== SUMMARY ===\n", res["summary"])

    print("\n=== SOURCES ===")
    for s in res.get("sources", []):
        # s: AnswerSource(source, page, score, preview)
        pg = f"|{s.page}" if s.page is not None else ""
        print(f"- {s.source}{pg}  (score={s.score})")
    print()

# -----------------------------------------------------------
# 8) Test pamięci — pytanie o historię rozmowy
# -----------------------------------------------------------
res = answerer.run(
    question="Na podstawie naszej historii rozmowy napisz, czym jest zainteresowany użytkownik w kontekście Mooff.",
    summarize=False,
    stream=False
)
print("\n=== MEMORY TEST ===\n", res["answer"])

if memory is not None:
    print("\n=== MEMORY DUMP ===")
    for m in memory.get_all():
        print(m)


# LLM + RAG + Memory

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

from intergrax.rag.embedding_manager import IntergraxEmbeddingManager
from intergrax.rag.rag_retriever import IntergraxRagRetriever
from intergrax.rag.vectorstore_manager import IntergraxVectorstoreManager, VSConfig
from intergrax.rag.re_ranker import IntergraxReRanker, ReRankerConfig
from intergrax.rag.rag_answerer import IntergraxAnswererConfig, IntergraxRagAnswerer
from intergrax.llm_adapters import LLMAdapterRegistry
from intergrax.llm.conversational_memory import IntergraxConversationalMemory
from langchain_ollama import ChatOllama
import intergrax.system_prompts as prompts

cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs",
    chroma_persist_directory="chroma_db/intergrax_docs_v1",
)
store = IntergraxVectorstoreManager(config=cfg, verbose=False)


embed_manager = IntergraxEmbeddingManager(
    verbose=False,
    provider="ollama",
    model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16:latest",
    assume_ollama_dim=1536
)

reranker = IntergraxReRanker(
    embedding_manager=embed_manager,
    config=ReRankerConfig(
        use_score_fusion=True,
        fusion_alpha=0.4,
        normalize="minmax",
        doc_batch_size=256
    ),
    verbose=True
)

retriever = IntergraxRagRetriever(store, embed_manager, verbose=False)


# create answerer configuration
cfg = IntergraxAnswererConfig(
    top_k=10,
    min_score=0.15,
    re_rank_k=5,
    max_context_chars=12000,
)
cfg.system_instructions = prompts.default_rag_system_instruction()
cfg.system_context_template = "Użyj następującego kontekstu aby odpowiedzieć na pytanie użytkownika: {context}"


llm = LLMAdapterRegistry.create(name="ollama", chat= ChatOllama(model="llama3.1:latest"))

memory = IntergraxConversationalMemory()

# create rag answerer (retriever + LLM)
answerer = IntergraxRagAnswerer(
    retriever=retriever,
    llm=llm,
    reranker=reranker,  
    config=cfg,
    memory=memory,
    verbose=False,
)

questions = [
    "Jakie unikalne przewagi na tle konkurencji posiada Mooff? Odpowiedź bardzo dokładnie używając nie mniej niż 1000 słów.",
    "Czym są wirtualne targi na platformie mooff ?",    
]


for question in questions:
    # create answer
    print("QUESTION: ", question)
    res = answerer.run(
        question=question,
        stream=False,
        summarize=True,
    )

    print("ANSWER: ", res['answer'])
    print()
    

res = answerer.run(
    question="Na podstawie historii konwersacji napisz czym zainteresowany jest użytkownik jeżeli chodzi o mooffa ?",
    summarize=False,
    stream=False
)
print("MEMORY TEST: ", res['answer'])
print()

if memory is not None:
    print("MEMORY:")
    # print(memory.get_all())
    for m in memory.get_all():
        print(m)

# Tool agent

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

from intergrax.llm.conversational_memory import IntergraxConversationalMemory, ChatMessage
from intergrax.llm_adapters import LLMAdapterRegistry
from intergrax.tools.tools_agent import IntergraxToolsAgent, ToolsAgentConfig
from intergrax.tools.tools_base import ToolRegistry, ToolBase
from langchain_ollama import ChatOllama
from pydantic import BaseModel, Field
import math

# ---------- WEATHER TOOL ----------
class WeatherArgs(BaseModel):
    city: str = Field(..., description="City name, e.g. 'Warsaw'")

class WeatherTool(ToolBase):
    name = "get_weather"
    description = "Returns current weather for a city. Use only for queries about weather/temperature/conditions in a city."
    schema_model = WeatherArgs

    def run(self, **kwargs):
        city = kwargs["city"]
        # demo output
        return {"city": city, "tempC": 12.3, "summary": "partly cloudy"}

# ---------- CALCULATOR TOOL ----------
class CalcArgs(BaseModel):
    expression: str = Field(
        ...,
        description="A safe arithmetic expression to evaluate, e.g. '235*17', '2*(3+4)'. No variables."
    )

class CalcTool(ToolBase):
    name = "calc_expression"
    description = "Safely evaluates a basic arithmetic expression (integers/floats, + - * / parentheses). Use for math/calculation questions."
    schema_model = CalcArgs

    def run(self, **kwargs):
        expr = kwargs["expression"]
        # Minimalny, bezpieczny eval – tylko do znaków matematycznych
        allowed = "0123456789.+-*/() "
        if not all(ch in allowed for ch in expr):
            raise ValueError("Unsupported characters in expression.")
        # Oblicz
        try:
            result = eval(expr, {"__builtins__": {}}, {})
        except Exception as e:
            raise ValueError(f"Invalid expression: {e}")
        return {"expression": expr, "result": result}

# ---------- AGENT SETUP ----------
memory = IntergraxConversationalMemory()
tools = ToolRegistry()
tools.register(WeatherTool())
tools.register(CalcTool())

# OLLAMA (planner JSON) – lub odkomentuj OpenAI dla native tools
llm = LLMAdapterRegistry.create(name="ollama", chat=ChatOllama(model="llama3.1:latest"))

# from openai import OpenAI
# client = OpenAI()
# llm = LLMAdapterRegistry.create(name="openai", client=client, model="gpt-4o-mini")

agent = IntergraxToolsAgent(
    llm=llm,
    tools=tools,
    memory=memory,
    config=ToolsAgentConfig(),
    verbose=True,    
)


# ---------- TEST 1: pogoda -> powinien wybrać get_weather ----------
res1 = agent.run("Jaka jest pogoda i temperatura w Warszawie?", context=None)
print("[ANS 1]", res1["answer"])
print("[TOOLS 1]", res1["tool_traces"])

# ---------- TEST 2: obliczenia -> powinien wybrać calc_expression ----------
res2 = agent.run("Policz 235*17 i podaj sam wynik.", context=None)
print("[ANS 2]", res2["answer"])
print("[TOOLS 2]", res2["tool_traces"])


# Output structure + Tools

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

from intergrax.llm.conversational_memory import IntergraxConversationalMemory
from intergrax.llm_adapters import LLMAdapterRegistry
from intergrax.tools.tools_agent import IntergraxToolsAgent, ToolsAgentConfig
from intergrax.tools.tools_base import ToolRegistry, ToolBase
from langchain_ollama import ChatOllama
from pydantic import BaseModel, Field

# ---------- SCHEMA ----------
class WeatherAnswer(BaseModel):
    """Structured representation of a weather query result."""
    city: str = Field(
        ...,
        description="City name for which the weather information applies. Example: 'Warsaw'."
    )
    tempC: float = Field(
        ...,
        description="Current temperature in degrees Celsius for the specified city."
    )
    summary: str = Field(
        ...,
        description="Short text summary of current weather conditions, e.g. 'partly cloudy', 'sunny', or 'light rain'."
    )

# ---------- TOOL ----------
class WeatherArgs(BaseModel):
    city: str = Field(..., description="City name, e.g. 'Warsaw'")

class WeatherTool(ToolBase):
    name = "get_weather"
    description = "Returns current weather for a city."
    schema_model = WeatherArgs

    def run(self, **kwargs):
        city = kwargs["city"]
        return {"city": city, "tempC": 12.3, "summary": "partly cloudy"}

# ---------- SETUP ----------
memory = IntergraxConversationalMemory()
tools = ToolRegistry()
tools.register(WeatherTool())

llm = LLMAdapterRegistry.create(name="ollama", chat=ChatOllama(model="llama3.1:latest"))

agent = IntergraxToolsAgent(
    llm=llm,
    tools=tools,
    memory=memory,
    config=ToolsAgentConfig(),
    verbose=False,
)

# ---------- UŻYCIE ----------
res = agent.run("Jaka jest pogoda w Warszawie?", context=None, output_model=WeatherAnswer)

print("[ANS]", res["answer"])
print("[OUTPUT_STRUCTURE]", res["output_structure"])      # ← obiekt WeatherAnswer
print("[AS DICT]", res["output_structure"].model_dump())  # np. {'city': 'Warsaw', 'tempC': 12.3, 'summary': 'partly cloudy'}


# Output structure + RAG

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

from intergrax.rag.embedding_manager import IntergraxEmbeddingManager
from intergrax.rag.rag_retriever import IntergraxRagRetriever
from intergrax.rag.vectorstore_manager import IntergraxVectorstoreManager, VSConfig
from intergrax.rag.re_ranker import IntergraxReRanker, ReRankerConfig
from intergrax.rag.rag_answerer import IntergraxAnswererConfig, IntergraxRagAnswerer
from intergrax.llm_adapters import LLMAdapterRegistry
from intergrax.llm.conversational_memory import IntergraxConversationalMemory
from langchain_ollama import ChatOllama
import intergrax.system_prompts as prompts
from dotenv import load_dotenv

load_dotenv()

# --- (opcjonalnie) Pydantic dla structured output ---
try:
    from pydantic import BaseModel, Field
except Exception:
    class BaseModel: ...
    def Field(*a, **k): return None  # fallback

# -------------------------
# SCHEMA do structured output
# -------------------------
class ExecSummary(BaseModel):
    """Zwięzła, ustrukturyzowana odpowiedź RAG (podsumowanie dla kadry)."""
    answer: str = Field(..., description="Krótka, klarowna odpowiedź bazująca wyłącznie na kontekście.")
    bullets: list[str] = Field(..., description="Lista 3-6 kluczowych punktów streszczenia.")
    citations: list[str] = Field(
        default_factory=list,
        description="Identyfikatory źródeł użytych w odpowiedzi (np. nazwa pliku lub tytuł)."
    )

# -------------------------
# VectorStore & embedding
# -------------------------
cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs",
    chroma_persist_directory="chroma_db/intergrax_docs_v1",
)
store = IntergraxVectorstoreManager(config=cfg, verbose=False)

embed_manager = IntergraxEmbeddingManager(
    verbose=False,
    provider="ollama",
    model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16:latest",
    assume_ollama_dim=1536
)

reranker = IntergraxReRanker(
    embedding_manager=embed_manager,
    config=ReRankerConfig(
        use_score_fusion=True,
        fusion_alpha=0.4,
        normalize="minmax",
        doc_batch_size=256
    ),
    verbose=True
)

retriever = IntergraxRagRetriever(store, embed_manager, verbose=False)

# -------------------------
# Answerer config
# -------------------------
cfg = IntergraxAnswererConfig(
    top_k=10,
    min_score=0.15,
    re_rank_k=5,
    max_context_chars=12000,
)
cfg.system_instructions = prompts.default_rag_system_instruction()
cfg.system_context_template = "Użyj następującego kontekstu aby odpowiedzieć na pytanie użytkownika: {context}"

# -------------------------
# LLM: OLLAMA
# (Adapter powinien wspierać generate_structured → np. przez tryb JSON + walidację)
# -------------------------
llm = LLMAdapterRegistry.create(name="ollama", chat=ChatOllama(model="llama3.1:latest"))
# from openai import OpenAI
# client = OpenAI()
# llm = LLMAdapterRegistry.create(name="openai", client=client, model="gpt-4o-mini")

memory = IntergraxConversationalMemory()

answerer = IntergraxRagAnswerer(
    retriever=retriever,
    llm=llm,
    reranker=reranker,
    config=cfg,
    memory=memory,
    verbose=False,
)

# -------------------------
# PYTANIE 1: ustrukturyzowana odpowiedź (output_model=ExecSummary)
# -------------------------
q1 = "Jakie unikalne przewagi na tle konkurencji posiada Mooff? Opisz jak najdokładniej. Użyj minimum 1000 słów. Użyj przekazanego modelu output_model do wygenerowania odpowiedzi."
print("QUESTION (structured):", q1)
res1 = answerer.run(
    question=q1,
    stream=False,
    summarize=False,           # summary niepotrzebne – mamy już strukturę
    output_model=ExecSummary,  # <<< wymuszenie schematu (zwrócone w 'output_structure')
)

# answer = tekst, output_structure = instancja ExecSummary lub None
print("ANSWER (text):", res1["answer"])
print("OUTPUT_STRUCTURE:", res1["output_structure"])
print("SOURCES:", [f"{s.source}|{s.page}" if s.page else s.source for s in res1["sources"]])
print()



# Chat Agent

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

from langchain_ollama import ChatOllama
import intergrax.system_prompts as prompts
from intergrax.llm_adapters import LLMAdapterRegistry
from intergrax.rag.embedding_manager import IntergraxEmbeddingManager
from intergrax.rag.rag_retriever import IntergraxRagRetriever
from intergrax.rag.rag_answerer import IntergraxRagAnswerer, IntergraxAnswererConfig
from intergrax.llm.conversational_memory import IntergraxConversationalMemory
from intergrax.tools.tools_base import ToolRegistry, ToolBase
from intergrax.rag.vectorstore_manager import VSConfig, IntergraxVectorstoreManager
from intergrax.rag.re_ranker import ReRankerConfig, IntergraxReRanker
from intergrax.chat_agent import ChatAgentConfig, RagComponent, IntergraxChatAgent
from pydantic import BaseModel, Field



llm = LLMAdapterRegistry.create(name="ollama", chat=ChatOllama(model="llama3.1:latest"))
memory = IntergraxConversationalMemory()


# ---------- WEATHER TOOL ----------
class WeatherArgs(BaseModel):
    city: str = Field(..., description="City name, e.g. 'Warsaw'")

class WeatherTool(ToolBase):
    name = "get_weather"
    description = "Returns current weather for a city. Use only for queries about weather/temperature/conditions in a city."
    schema_model = WeatherArgs

    def run(self, **kwargs):
        city = kwargs["city"]
        # demo output
        return {"city": city, "tempC": 12.3, "summary": "partly cloudy"}



# Tools (opcjonalnie)
tools = ToolRegistry()
tools.register(WeatherTool())


cfg = VSConfig(
    provider="chroma",
    collection_name="intergrax_docs",
    chroma_persist_directory="chroma_db/intergrax_docs_v1",
)
store = IntergraxVectorstoreManager(config=cfg, verbose=False)


embed_manager = IntergraxEmbeddingManager(
    verbose=False,
    provider="ollama",
    model_name="rjmalagon/gte-qwen2-1.5b-instruct-embed-f16:latest",
    assume_ollama_dim=1536
)

reranker = IntergraxReRanker(
    embedding_manager=embed_manager,
    config=ReRankerConfig(
        use_score_fusion=True,
        fusion_alpha=0.4,
        normalize="minmax",
        doc_batch_size=256
    ),
    verbose=True
)

retriever = IntergraxRagRetriever(store, embed_manager, verbose=False)


# create answerer configuration
cfg = IntergraxAnswererConfig(
    top_k=10,
    min_score=0.15,
    re_rank_k=5,
    max_context_chars=12000,
)
cfg.system_instructions = prompts.default_rag_system_instruction()
cfg.system_context_template = "Użyj następującego kontekstu aby odpowiedzieć na pytanie użytkownika: {context}"


# create rag answerer (retriever + LLM)
answerer_docs = IntergraxRagAnswerer(
    retriever=retriever,
    llm=llm,
    reranker=reranker,  
    config=cfg,
    memory=None,
    verbose=False,
)

rag_docs = RagComponent(
    name="intergrax_docs",
    answerer=answerer_docs,
    description=(
        "Ten komponent obsługuje wszystkie dane i informacje związane z regulaminami, "
        "politykami prywatności, zasadami bezpieczeństwa oraz dokumentacją wewnętrzną Mooff. "
        "Użyj go, gdy trzeba sprawdzić zgodność działania z politykami firmy, znaleźć postanowienia regulaminu "
        "lub odpowiedzieć na pytania bazujące na oficjalnych dokumentach i politykach."
    ),
    priority=10,
)

agent = IntergraxChatAgent(
    llm=llm,
    memory=memory,
    tools=tools,
    rag_components=[rag_docs],
    config=ChatAgentConfig(verbose=True),
)


for question in ["jaka jest pogoda w Warszawie ?", "czym są wirtualne targi mooff - podaj szczegółowe wyjaśnienie.", "policz 22 *3 i podaj wynik"]:    
    print(question)
    res1 = agent.run(question=question)
    print("Route: ",res1["route"])
    print("Rag: ",res1['rag_component'])
    print("Tool: ",res1['tool_traces'])
    print("Answer: ",res1['answer'])
    print("=============================================")


